In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import datasets
import matplotlib.pyplot as plt

test=pd.read_csv('users_behavior.csv')

In [3]:
test.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
Заменим тип объекта messages на int

In [7]:
test['messages'] = test['messages'].astype('int')

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int32  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(3), int32(1), int64(1)
memory usage: 113.1 KB


In [10]:
test.duplicated().sum()

0

In [16]:
design_matrix = test.loc[:,~test.columns.isin(["is_ultra"])]
target_vector = test["is_ultra"]

In [43]:
from sklearn.model_selection import train_test_split

(design_matrix_train, design_matrix_test,target_vector_train,target_vector_test) = train_test_split(design_matrix, target_vector, test_size = 0.3, random_state = 47)

Разделим выборку

In [44]:
(design_matrix_valid, design_matrix_test, target_vector_valid, target_vector_test) = train_test_split(design_matrix_test, target_vector_test, test_size = 0.7, random_state = 47 )

In [56]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm


best_DT = None
best_accuracy_DT = 0
best_depth_DT = 0
for depth in tqdm(range(2,50)):
    DT = DecisionTreeClassifier(random_state = 88, max_depth = depth)
    DT.fit(design_matrix_train,target_vector_train)
    prediction_valid_DT = DT.predict(design_matrix_valid)
    accuracy_DT = accuracy_score(prediction_valid_DT, target_vector_valid)
    if best_accuracy_DT < accuracy_DT:
        best_DT = DT
        best_accuracy_DT = accuracy_DT
        best_depth_DT = depth

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 49.91it/s]


In [60]:
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier

best_RF = None
best_accuracy_RF = 0
best_est_RF = 0
best_depth_RF = 0
for est in tqdm(range(2,51)):
    for depth in range(2,25):
        RF = RandomForestClassifier(random_state = 47 ,n_estimators = est, max_depth = depth)
        RF.fit(design_matrix_train,target_vector_train)
        prediction_valid_RF = RF.predict(design_matrix_valid)
        accuracy_RF = accuracy_score(prediction_valid_RF, target_vector_valid)
        if best_accuracy_RF < accuracy_RF:
            best_RF = RF
            best_depth_RF = depth
            best_est_RF = est
            best_accuracy_RF = accuracy_RF

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [02:44<00:00,  3.35s/it]


In [57]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(random_state = 47, max_iter= 1000,tol=1e-5,solver = 'lbfgs' )
LR.fit(design_matrix_train,target_vector_train)
prediction_valid_LR = LR.predict(design_matrix_valid)
accuracy_LR = accuracy_score(prediction_valid_LR, target_vector_valid)

In [58]:
print(f"Logistic Regression: {accuracy_LR}")
print(f"Random Forest: {best_accuracy_RF}")
print(f"Decision Tree: {best_accuracy_DT}")

Logistic Regression: 0.7058823529411765
Random Forest: 0.8166089965397924
Decision Tree: 0.8027681660899654


In [59]:
prediction_test_LR = LR.predict(design_matrix_test)
prediction_test_RF = best_RF.predict(design_matrix_test)
prediction_test_DT = best_DT.predict(design_matrix_test)

accuracy_LR_test = accuracy_score(prediction_test_LR, target_vector_test)
accuracy_RF_test = accuracy_score(prediction_test_RF, target_vector_test)
accuracy_DT_test = accuracy_score(prediction_test_DT, target_vector_test)

print(f"Logistic Regression: {accuracy_LR_test}")
print(f"Random Forest: {accuracy_RF_test}")
print(f"Decision Tree: {accuracy_DT_test}")

Logistic Regression: 0.7130177514792899
Random Forest: 0.8017751479289941
Decision Tree: 0.7810650887573964
